Copiar dataset en una carpeta `data`

In [1]:
# Create the directory if it doesn't exist
!mkdir ../data

# Download files using curl
!curl -o ../data/green_tripdata_2024-01.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-01.parquet
!curl -o ../data/green_tripdata_2024-02.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-02.parquet

La sintaxis del comando no es correcta.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1330k  100 1330k    0     0   855k      0  0:00:01  0:00:01 --:--:--  862k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1253k  100 1253k    0     0  2618k      0 --:--:-- --:--:-- --:--:-- 2678k


Importar las librerías necesarias y definir función para importar los datos

In [1]:
import pickle
import pandas as pd
from sklearn.metrics import  root_mean_squared_error
from sklearn.feature_extraction import  DictVectorizer
from sklearn.linear_model import Lasso, Ridge, LinearRegression

In [2]:
def read_dataframe(filename):

    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [3]:
df_train = read_dataframe('../data/green_tripdata_2024-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2024-02.parquet')

Feature Engineering

In [4]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

One Hot Encoding

In [5]:
categorical = ['PU_DO']  #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [6]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

Definir el `tracking URI` y el nombre del experimento

In [7]:
import dagshub
import mlflow


dagshub.init(url="https://dagshub.com/G4ll4rd0/nyc-taxi-time-prediction", mlflow=True)

MLFLOW_TRACKING_URI = mlflow.get_tracking_uri()

print(MLFLOW_TRACKING_URI)

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(experiment_name="nyc-taxi-experiment")

Accessing as G4ll4rd0

Initialized MLflow to track repo "G4ll4rd0/nyc-taxi-time-prediction"

Repository G4ll4rd0/nyc-taxi-time-prediction initialized!

https://dagshub.com/G4ll4rd0/nyc-taxi-time-prediction.mlflow


<Experiment: artifact_location='mlflow-artifacts:/63ce2eff028d42fda8747e5857239051', creation_time=1726773560535, experiment_id='0', last_update_time=1726773560535, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

Definir los `dataset` como objetos de `mlflow` para poderlos trackear

In [8]:
training_dataset = mlflow.data.from_numpy(X_train.data, targets=y_train, name="green_tripdata_2024-01")
validation_dataset = mlflow.data.from_numpy(X_val.data, targets=y_val, name="green_tripdata_2024-02")

### Subir los dataset al storage que nos brinda `dagshub`

In [10]:
from dagshub import get_repo_bucket_client
import os
# Get a boto3.client object
s3 = get_repo_bucket_client("G4ll4rd0/nyc-taxi-time-prediction")


basepath = os.path.abspath('../data')

for file in os.listdir(basepath):
    filepath = os.path.join(basepath, file)
    # Upload file
    s3.upload_file(
        Bucket = "nyc-taxi-time-prediction",  # name of the repo
        Filename = filepath,  # local path of file to upload
        Key = file,  # remote path where to upload the file
    )

Client created. Use the name of the repo (nyc-taxi-time-prediction) as the name of the bucket

Ahora vamos a entrenar un modelo `xgboost`


In [9]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
import pathlib

Definir los `dataset` a trabajar.

In [10]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

Definir la función objetivo

In [11]:
def objective(params):
    with mlflow.start_run(nested=True):
         
        # Tag model
        mlflow.set_tag("model_family", "xgboost")
        
        # Log parameters
        mlflow.log_params(params)
        
        # Train model
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, 'validation')],
            early_stopping_rounds=10
        )
        
        # Log xgboost model with artifact_path
        mlflow.xgboost.log_model(booster, artifact_path="model")

        # Predict in the val dataset
        y_pred = booster.predict(valid)
        
        # Calculate metric
        rmse = root_mean_squared_error(y_val, y_pred)
        
        # Log performance metric
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

Definir el espacio de búsqueda

In [12]:
mlflow.xgboost.autolog()

with mlflow.start_run(run_name="Xgboost Hyper-parameter Optimization", nested=True):
    search_space = {
        'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
        'learning_rate': hp.loguniform('learning_rate', -3, 0),
        'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
        'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
        'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
        'objective': 'reg:squarederror',
        'seed': 42
    }

    best_params = fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=10,
        trials=Trials()
    )
    best_params["max_depth"] = int(best_params["max_depth"])
    best_params["seed"] = 42
    best_params["objective"] = "reg:squarederror"

    mlflow.log_params(best_params)

    # Log tags
    mlflow.set_tags(
        tags={
            "project": "NYC Taxi Time Prediction Project",
            "optimizer_engine": "hyper-opt",
            "model_family": "xgboost",
            "feature_set_version": 1,
        }
    )

    # Log a fit model instance
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=100,
        evals=[(valid, 'validation')],
        early_stopping_rounds=10
    )

    y_pred = booster.predict(valid)

    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    pathlib.Path("models").mkdir(exist_ok=True)
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

[0]	validation-rmse:8.67760                           
[1]	validation-rmse:8.28754                           
[2]	validation-rmse:7.93578                           
[3]	validation-rmse:7.62048                           
[4]	validation-rmse:7.33823                           
[5]	validation-rmse:7.08771                           
[6]	validation-rmse:6.86281                           
[7]	validation-rmse:6.66576                           
[8]	validation-rmse:6.48705                           
[9]	validation-rmse:6.33213                           
[10]	validation-rmse:6.19398                          
[11]	validation-rmse:6.07334                          
[12]	validation-rmse:5.96510                          
[13]	validation-rmse:5.87162                          
[14]	validation-rmse:5.78835                          
[15]	validation-rmse:5.71651                          
[16]	validation-rmse:5.65275                          
[17]	validation-rmse:5.59605                          
[18]	valid

2024/09/19 18:20:39 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/09/19 18:20:39 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\Rafael\OneDrive - ITESO\2024.3 Otoño\Proyecto de Ciencia de Datos\nyc-taxi-time-prediction\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:20:39] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/09/19 18:21:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

c:\Users\Rafael\OneDrive - ITESO\2024.3 Otoño\P

[0]	validation-rmse:7.40902                                                     
[1]	validation-rmse:6.41306                                                     
[2]	validation-rmse:5.85430                                                     
[3]	validation-rmse:5.54858                                                     
[4]	validation-rmse:5.39451                                                     
[5]	validation-rmse:5.30482                                                     
[6]	validation-rmse:5.25883                                                     
[7]	validation-rmse:5.23209                                                     
[8]	validation-rmse:5.21253                                                     
[9]	validation-rmse:5.20342                                                     
[10]	validation-rmse:5.19739                                                    
[11]	validation-rmse:5.19204                                                    
[12]	validation-rmse:5.18777

2024/09/19 18:22:38 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/09/19 18:22:38 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\Rafael\OneDrive - ITESO\2024.3 Otoño\Proyecto de Ciencia de Datos\nyc-taxi-time-prediction\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:22:38] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/09/19 18:22:47 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

c:\Users\Rafael\OneDrive - ITESO\2024.3 Otoño\P

[0]	validation-rmse:7.34627                                                     
[1]	validation-rmse:6.38447                                                     
[2]	validation-rmse:5.89811                                                     
[3]	validation-rmse:5.64771                                                     
[4]	validation-rmse:5.51140                                                     
[5]	validation-rmse:5.44979                                                     
[6]	validation-rmse:5.40580                                                     
[7]	validation-rmse:5.38392                                                     
[8]	validation-rmse:5.37095                                                     
[9]	validation-rmse:5.36219                                                     
[10]	validation-rmse:5.35818                                                    
[11]	validation-rmse:5.35669                                                    
[12]	validation-rmse:5.35334

...

2024/09/19 18:30:28 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/09/19 18:30:28 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\Rafael\OneDrive - ITESO\2024.3 Otoño\Proyecto de Ciencia de Datos\nyc-taxi-time-prediction\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:30:28] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/09/19 18:30:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

c:\Users\Rafael\OneDrive - ITESO\2024.3 Otoño\P

[0]	validation-rmse:7.49176                                                    
[1]	validation-rmse:6.50018                                                    
[2]	validation-rmse:5.92112                                                    
[3]	validation-rmse:5.59945                                                    
[4]	validation-rmse:5.42470                                                    
[5]	validation-rmse:5.32840                                                    
[6]	validation-rmse:5.28203                                                    
[7]	validation-rmse:5.25424                                                    
[8]	validation-rmse:5.23508                                                    
[9]	validation-rmse:5.22607                                                    
[10]	validation-rmse:5.22162                                                   
[11]	validation-rmse:5.21661                                                   
[12]	validation-rmse:5.21154            

2024/09/19 18:32:23 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/09/19 18:32:23 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\Rafael\OneDrive - ITESO\2024.3 Otoño\Proyecto de Ciencia de Datos\nyc-taxi-time-prediction\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:32:23] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/09/19 18:32:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

c:\Users\Rafael\OneDrive - ITESO\2024.3 Otoño\P

100%|██████████| 10/10 [16:01<00:00, 96.16s/trial, best loss: 5.179563303705733]
[0]	validation-rmse:8.67760
[1]	validation-rmse:8.28754
[2]	validation-rmse:7.93578
[3]	validation-rmse:7.62048
[4]	validation-rmse:7.33823
[5]	validation-rmse:7.08771
[6]	validation-rmse:6.86281
[7]	validation-rmse:6.66576
[8]	validation-rmse:6.48705
[9]	validation-rmse:6.33213
[10]	validation-rmse:6.19398
[11]	validation-rmse:6.07334
[12]	validation-rmse:5.96510
[13]	validation-rmse:5.87162
[14]	validation-rmse:5.78835
[15]	validation-rmse:5.71651
[16]	validation-rmse:5.65275
[17]	validation-rmse:5.59605
[18]	validation-rmse:5.54898
[19]	validation-rmse:5.50559
[20]	validation-rmse:5.46924
[21]	validation-rmse:5.43625
[22]	validation-rmse:5.40723
[23]	validation-rmse:5.38192
[24]	validation-rmse:5.36072
[25]	validation-rmse:5.34125
[26]	validation-rmse:5.32462
[27]	validation-rmse:5.30812
[28]	validation-rmse:5.29615
[29]	validation-rmse:5.28361
[30]	validation-rmse:5.27315
[31]	validation-rmse:5.26329
[

2024/09/19 18:34:36 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/09/19 18:34:36 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\Rafael\OneDrive - ITESO\2024.3 Otoño\Proyecto de Ciencia de Datos\nyc-taxi-time-prediction\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:34:36] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
2024/09/19 18:34:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/09/19 18:34:55 INFO mlflow.tracking._tracking

In [13]:
best_params

{'learning_rate': np.float64(0.07174919146863269),
 'max_depth': 67,
 'min_child_weight': np.float64(1.7611277817991904),
 'reg_alpha': np.float64(0.016274588255319838),
 'reg_lambda': np.float64(0.0223176786967516),
 'seed': 42,
 'objective': 'reg:squarederror'}

Ahora vamos a registrar el mejor modelo en el `model registry` y usarlo para hacer predicciones

In [14]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri=run_uri,
    name="nyc-taxi-model"
)

Successfully registered model 'nyc-taxi-model'.
2024/09/19 18:36:04 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-model, version 1
Created version '1' of model 'nyc-taxi-model'.


In [15]:
from datetime import datetime
from mlflow import MlflowClient

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
client.update_registered_model(
    name="nyc-taxi-model",
    description="Model registry for the NYC Taxi Time Prediction Project",
)

new_alias = "champion"
date = datetime.today()
model_version = "1"

# create "champion" alias for version 1 of model "nyc-taxi-model"
client.set_registered_model_alias(
    name="nyc-taxi-model",
    alias=new_alias,
    version=model_version
)

client.update_model_version(
    name="nyc-taxi-model",
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_alias} on {date}",
)

<ModelVersion: aliases=['champion'], creation_timestamp=1726792565762, current_stage='None', description='The model version 1 was transitioned to champion on 2024-09-19 18:36:07.585301', last_updated_timestamp=1726792569112, name='nyc-taxi-model', run_id='51fdab30ac6d4e9297625835169f2004', run_link='', source='mlflow-artifacts:/63ce2eff028d42fda8747e5857239051/51fdab30ac6d4e9297625835169f2004/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [16]:
import mlflow.pyfunc

model_name = "nyc-taxi-model"
alias = "champion"

model_uri = f"models:/{model_name}@{alias}"

champion_version = mlflow.pyfunc.load_model(
    model_uri=model_uri
)

champion_version.predict(X_val)

array([19.052462, 28.202484,  9.465672, ..., 46.640884, 14.011147,
       19.779663], dtype=float32)